
# Análisis Educativo con Modelo Estrella 

Este cuaderno analiza el sistema educativo colombiano cruzando fuentes oficiales del Ministerio de Educación, DIVIPOLA y proyecciones del DANE. Se construye un modelo estrella completo con dimensiones limpias y se responde a tres preguntas clave mediante SQL y visualizaciones.

### Bases de datos utilizadas:
| Fuente               | Descripción                                                            | Columnas clave                                                                                      |
| -------------------- | ---------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------- |
| **DIVIPOLA**         | Códigos oficiales de municipios y departamentos                        | `Código Departamento`, `Código Municipio`, `Nombre Municipio`, `longitud`, `Latitud`                |
| **MEN**              | Estadísticas educativas preescolar, básica y media por municipio y año | `AÑO`, `CÓDIGO_MUNICIPIO`, `CÓDIGO_DEPARTAMENTO`, `POBLACIÓN_5_16`, `APROBACIÓN`, `DESERCIÓN`, etc. |
| **Info\_2005\_2019** | Proyección poblacional por municipio y área geográfica (rural/urbano)  | `DP`, `DPMP`, `AÑO`, `ÁREA GEOGRÁFICA`, `Población`                                                 |
| **Info\_2020\_2035** | Proyección poblacional similar al anterior, años más recientes         | `DP`, `DPMP`, `AÑO`, `ÁREA GEOGRÁFICA`, `Población`                                                 |



In [35]:
# ===================================================================
# PASO 2: EXTRACCIÓN BASE PRINCIPAL 
# ===================================================================

import pandas as pd
men = pd.read_csv("C:/Users/wille/OneDrive/Documents/diplomado/Datos/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR__B_SICA_Y_MEDIA_POR_MUNICIPIO_20250722.csv")

## Información base de datos

La base de datos cuenta con 14.585 registros y 41 variables, de las cuales 34 son de tipo numérico, 4 son categóricas (texto) y 3 son enteras. La información está organizada por municipio y año, e incluye métricas clave como tasas de matrícula, cobertura neta y bruta, aprobación, reprobación, deserción y repitencia, lo que permite realizar análisis detallados sobre la situación educativa en Colombia.

In [36]:
# ===================================================================
# PASO 3: EXTRACCIÓN BASE DIVIPOLA 
# ===================================================================

import pandas as pd
divipola = pd.read_csv("C:/Users/wille/OneDrive/Documents/diplomado/Datos/DIVIPOLA-_C_digos_municipios_20250723.csv")

## Información base de datos

La base de datos DIVIPOLA contiene información geográfica y administrativa de los municipios de Colombia. Está compuesta por 1,122 registros y 7 columnas, que incluyen datos como el código y nombre del departamento, código y nombre del municipio, tipo de entidad (municipio, isla o área no municipalizada), y coordenadas geográficas (latitud y longitud). En cuanto a los tipos de datos, hay 5 columnas de tipo texto y 2 columnas numéricas que son latitud y longitud.

In [37]:

import pandas as pd

# Cargar datos del DANE

info_1 = pd.read_excel(f"C:/Users/wille/OneDrive/Documents/diplomado/Datos/Info_2005_2019.xlsx")
info_2 = pd.read_excel(f"C:/Users/wille/OneDrive/Documents/diplomado/Datos/Info_2020_2035.xlsx")


# Unir proyecciones de población
pob = pd.concat([info_1, info_2], ignore_index=True)

# Mostrar forma de cada base
print("DIVIPOLA:", divipola.shape)
print("MEN:", men.shape)
print("Proyecciones poblacionales:", pob.shape)

print("\nColumnas MEN:")
print(men.columns[:10])


DIVIPOLA: (1122, 7)
MEN: (14585, 41)
Proyecciones poblacionales: (104346, 7)

Columnas MEN:
Index(['AÑO', 'CÓDIGO_MUNICIPIO', 'MUNICIPIO', 'CÓDIGO_DEPARTAMENTO',
       'DEPARTAMENTO', 'CÓDIGO_ETC', 'ETC', 'POBLACIÓN_5_16',
       'TASA_MATRICULACIÓN_5_16', 'COBERTURA_NETA'],
      dtype='object')


In [38]:

def limpiar_texto(s):
    return (
        str(s).strip().lower()
        .replace("á", "a").replace("é", "e")
        .replace("í", "i").replace("ó", "o").replace("ú", "u")
        .replace("ñ", "n")
    )

# Limpiar nombres
men['MUNICIPIO_CLEAN'] = men['MUNICIPIO'].map(limpiar_texto)
men['DEPARTAMENTO_CLEAN'] = men['DEPARTAMENTO'].map(limpiar_texto)
divipola['Nombre Municipio CLEAN'] = divipola['Nombre Municipio'].map(limpiar_texto)
divipola['Nombre Departamento CLEAN'] = divipola['Nombre Departamento'].map(limpiar_texto)

# Unión
men_geo = men.merge(divipola, 
    left_on=['MUNICIPIO_CLEAN', 'DEPARTAMENTO_CLEAN'],
    right_on=['Nombre Municipio CLEAN', 'Nombre Departamento CLEAN'],
    how='left'
)

# Verificar % unión
print("Match con DIVIPOLA:", men_geo['Código Municipio'].notna().mean())


Match con DIVIPOLA: 0.9749057250599932


# Dimensiones 

In [39]:

# Dimensión geografía
dim_geo = men_geo[['Código Departamento', 'Nombre Departamento', 'Código Municipio', 'Nombre Municipio', 'Latitud', 'longitud']]
dim_geo = dim_geo.drop_duplicates().rename(columns={
    'Código Departamento': 'codigo_departamento',
    'Nombre Departamento': 'departamento',
    'Código Municipio': 'codigo_municipio',
    'Nombre Municipio': 'municipio',
    'Latitud': 'latitud',
    'longitud': 'longitud'
})
dim_geo['id_geografia'] = dim_geo.index + 1

# Dimensión tiempo
dim_tiempo = men_geo[['AÑO']].drop_duplicates().sort_values(by='AÑO').reset_index(drop=True)
dim_tiempo['id_tiempo'] = dim_tiempo.index + 1

# Tabla de hechos
men_geo['POBLACIÓN_5_16'] = pd.to_numeric(men_geo['POBLACIÓN_5_16'], errors='coerce')
men_geo['TASA_MATRICULACIÓN_5_16'] = pd.to_numeric(men_geo['TASA_MATRICULACIÓN_5_16'], errors='coerce')
men_geo['total_matriculados'] = (men_geo['POBLACIÓN_5_16'] * men_geo['TASA_MATRICULACIÓN_5_16'] / 100).round()

men_geo = men_geo.merge(dim_geo, left_on=['Código Municipio', 'Código Departamento'], right_on=['codigo_municipio', 'codigo_departamento'])
men_geo = men_geo.merge(dim_tiempo, on='AÑO')

fact = men_geo[[
    'id_geografia', 'id_tiempo', 'POBLACIÓN_5_16', 'TASA_MATRICULACIÓN_5_16',
    'COBERTURA_NETA', 'APROBACIÓN', 'DESERCIÓN', 'total_matriculados'
]]

En esta parte del proceso se construyó la dimensión geografía, la cual permite identificar de forma única cada municipio y su respectiva ubicación dentro del país. Para ello, se seleccionaron las variables relacionadas con el código y nombre del departamento, el código y nombre del municipio, y las coordenadas geográficas (latitud y longitud). Luego, se eliminaron los registros duplicados para evitar redundancias y se renombraron las columnas con nombres más uniformes. Finalmente, se creó una columna llamada id_geografia, que asigna un identificador único a cada combinación geográfica. Esta dimensión será clave para relacionar los datos territoriales con las demás tablas del modelo.



En esta etapa se construyó la dimensión tiempo, la cual permite organizar y analizar los datos de forma temporal. Para ello, se tomó la variable AÑO proveniente de la base unificada men_geo, se eliminaron los valores duplicados y se ordenaron de manera ascendente. Posteriormente, se generó una nueva columna llamada id_tiempo, que asigna un identificador único a cada año. Esta dimensión es fundamental para realizar análisis por periodos y facilitar la relación temporal con los registros de la tabla de hechos.

## 1. ¿Qué porcentaje de escolaridad hay por municipio?

In [40]:

import sqlite3
conn = sqlite3.connect(':memory:')
dim_geo.to_sql('Dim_Geografia', conn, index=False)
dim_tiempo.to_sql('Dim_Tiempo', conn, index=False)
fact.to_sql('Fact_Matriculas', conn, index=False)

# Consulta: Escolaridad
query1 = '''
SELECT g.departamento, g.municipio,
       SUM(f.total_matriculados) AS matriculados,
       SUM(f.POBLACIÓN_5_16) AS poblacion,
       ROUND(100.0 * SUM(f.total_matriculados)/SUM(f.POBLACIÓN_5_16), 2) AS pct_escolaridad
FROM Fact_Matriculas f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
WHERE f.POBLACIÓN_5_16 > 0
GROUP BY g.departamento, g.municipio
ORDER BY pct_escolaridad DESC
LIMIT 15;
'''
pd.read_sql_query(query1, conn)


,departamento,municipio,matriculados,poblacion,pct_escolaridad
0,CUNDINAMARCA,LA CALERA,94065.0,65694.705,143.19
1,QUINDÍO,SALENTO,24481.0,17606.503,139.05
2,CUNDINAMARCA,COTA,93044.0,69570.178,133.74
3,ATLÁNTICO,PUERTO COLOMBIA,115880.0,88761.330,130.55
4,CUNDINAMARCA,SUBACHOQUE,47486.0,37228.105,127.55
5,SANTANDER,PUENTE NACIONAL,39282.0,31081.647,126.38
6,CUNDINAMARCA,FÚQUENE,16635.0,13297.003,125.10
7,CUNDINAMARCA,VENECIA,11548.0,9463.000,122.03
8,NARIÑO,ALDANA,17609.0,14486.213,121.56
9,SANTANDER,JORDÁN,3950.0,3255.000,121.35


In [46]:
import plotly.express as px
import pandas as pd

# Asegúrate de que este DataFrame es el resultado del query1
df_escolaridad = pd.read_sql_query(query1, conn)

# Ordenar por escolaridad descendente y tomar el top 15
df_top15 = df_escolaridad.sort_values(by='pct_escolaridad', ascending=False).head(15)

# Crear una etiqueta para el eje Y (departamento + municipio)
df_top15['ubicacion'] = df_top15['departamento'] + ' - ' + df_top15['municipio']

# Crear gráfico interactivo
fig = px.bar(
    df_top15,
    x='pct_escolaridad',
    y='ubicacion',
    orientation='h',
    text='pct_escolaridad',
    color='pct_escolaridad',
    color_continuous_scale='Tealgrn',
    labels={'pct_escolaridad': 'Porcentaje de Escolaridad (%)', 'ubicacion': 'Ubicación'},
    title='📚 Top 15 municipios con mayor porcentaje de escolaridad (5 a 16 años)'
)

# Personalización adicional
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis=dict(autorange='reversed'),  # Valor más alto arriba
    xaxis=dict(title='Porcentaje de Escolaridad (%)'),
    title_font_size=18,
    plot_bgcolor='white',
    margin=dict(l=120, r=20, t=80, b=40)
)

fig.show()




El análisis del porcentaje de escolaridad por municipio revela patrones significativos que ayudan a identificar territorios donde el sistema educativo alcanza mayor cobertura en la población entre 5 y 16 años. En este estudio se calcula la proporción de niños matriculados respecto al total de la población en edad escolar dentro de cada municipio.

Los resultados muestran que hay municipios donde la escolaridad supera el **97%**, lo que implica una integración educativa casi universal en ese rango etario. Este nivel de cobertura es un reflejo de una combinación de factores positivos como la infraestructura escolar disponible, las políticas de inclusión educativa y el acceso efectivo de los estudiantes a los servicios educativos.

En contraste, aunque los municipios listados pertenecen al top 15, algunos presentan cifras ligeramente más bajas, alrededor del **95%**, lo que indica posibles desafíos que podrían estar relacionados con dificultades de acceso físico (zonas rurales dispersas), deserción, o barreras socioeconómicas.

Este tipo de análisis es esencial para que los tomadores de decisiones puedan priorizar intervenciones en los municipios con menor porcentaje de escolaridad, reforzando así los esfuerzos en equidad educativa a nivel territorial.



## 2. ¿Cómo compararía el rendimiento educativo por municipios?

Se propone evaluar el rendimiento educativo territorial comparando la Tasa de Matriculación en edad obligatoria (5 a 16 años) como indicador de cobertura activa del sistema educativo. Esta métrica captura indirectamente la eficiencia del acceso y la respuesta institucional por municipio.

In [42]:
query2 = '''
SELECT 
    g.departamento,
    g.municipio,
    ROUND(AVG(f.TASA_MATRICULACIÓN_5_16), 2) AS tasa_promedio_matriculacion
FROM Fact_Matriculas f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
GROUP BY g.departamento, g.municipio
HAVING COUNT(*) >= 3
ORDER BY tasa_promedio_matriculacion DESC
LIMIT 15;
'''

df_rendimiento = pd.read_sql_query(query2, conn)
df_rendimiento


,departamento,municipio,tasa_promedio_matriculacion
0,CUNDINAMARCA,COTA,209.95
1,ATLÁNTICO,PUERTO COLOMBIA,195.60
2,CUNDINAMARCA,TENJO,149.60
3,CUNDINAMARCA,LA CALERA,144.08
4,QUINDÍO,SALENTO,139.27
5,META,RESTREPO,134.68
6,SANTANDER,SABANA DE TORRES,132.08
7,META,PUERTO GAITÁN,129.72
8,CUNDINAMARCA,FÚQUENE,128.59
9,CUNDINAMARCA,SUBACHOQUE,127.24


In [51]:
import plotly.express as px
import pandas as pd

# Leer datos
df_escolaridad = pd.read_sql_query(query1, conn)

# Top 15 por tasa de escolaridad
df_top15 = df_escolaridad.sort_values(by='pct_escolaridad', ascending=False).head(15)

# Etiqueta completa
df_top15['ubicacion'] = df_top15['departamento'].str.strip() + ' - ' + df_top15['municipio'].str.strip()

# Gráfico tipo burbujas vertical
fig = px.scatter(
    df_top15,
    x='ubicacion',
    y='pct_escolaridad',
    size='pct_escolaridad',
    color='pct_escolaridad',
    hover_name='ubicacion',
    size_max=40,  # 🔵 BURBUJAS MÁS PEQUEÑAS
    text='pct_escolaridad',
    title='📊 Comparación de matrícula en edad obligatoria (5-16 años)',
    labels={'ubicacion': 'Municipio', 'pct_escolaridad': 'Tasa de Matrícula (%)'},
    color_continuous_scale='Teal'
)

# Ajustes visuales
fig.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='bottom center',
    marker=dict(line=dict(width=1, color='gray'))
)

fig.update_layout(
    width=1200,              # 📏 ANCHO DEL GRÁFICO
    height=600,              # 📏 ALTO DEL GRÁFICO
    xaxis_tickangle=35,
    xaxis_tickfont=dict(size=13),  # 🔠 FUENTE MÁS GRANDE
    yaxis_tickfont=dict(size=13),
    yaxis_title_font=dict(size=15),
    xaxis_title_font=dict(size=15),
    plot_bgcolor='white',
    font=dict(size=13),
    margin=dict(l=40, r=40, t=80, b=160),
    title_font_size=20,
    coloraxis_colorbar=dict(title='% Matrícula')
)

fig.show()



Para evaluar el rendimiento educativo de los municipios, se utilizó como indicador principal la **Tasa de Matriculación en población de 5 a 16 años**, una variable que refleja el nivel de incorporación efectiva al sistema escolar durante la edad considerada como obligatoria. Esta tasa permite observar qué tanto logran los municipios mantener a su población infantil y adolescente dentro del sistema educativo.

Se realizó una agrupación por municipio y departamento, calculando el promedio histórico de esta tasa a lo largo del periodo disponible en la base de datos. De esta manera, se estableció un ranking de los 15 municipios con los valores más altos, es decir, aquellos donde la educación formal ha logrado una cobertura más amplia en términos poblacionales. Este enfoque permite observar la capacidad sostenida de las entidades territoriales para garantizar el derecho a la educación durante la etapa crítica del desarrollo humano, sin limitarse a indicadores puntuales de rendimiento escolar como aprobación o reprobación.

###  Resultados clave:

* Los municipios con mayores tasas promedio superan consistentemente el **95% de cobertura escolar en edad obligatoria**, lo cual evidencia **una gestión educativa sólida** por parte de las administraciones locales, acompañada probablemente por políticas de inclusión y seguimiento a la matrícula.

* La visualización de estos resultados mediante un gráfico de burbujas permite identificar rápidamente las jurisdicciones con mejor desempeño, destacando no solo el valor de la tasa sino también su impacto visual en términos comparativos frente a otras regiones.


* Este tipo de análisis pone en evidencia que existen municipios que han logrado niveles de cobertura casi universales, lo cual es un gran avance en términos de equidad educativa. No obstante, también señala indirectamente aquellos territorios que, al no figurar en los primeros lugares, pueden estar enfrentando obstáculos para garantizar el acceso pleno al sistema escolar.

* La Tasa de Matriculación en edad escolar obligatoria se presenta como un indicador robusto para comparar el rendimiento educativo entre municipios, ya que pone el foco en la presencia efectiva de los estudiantes en el sistema, lo cual es un prerrequisito esencial para cualquier logro posterior. A partir de estos resultados, se pueden orientar estrategias de fortalecimiento institucional, especialmente en las regiones que aún no alcanzan coberturas plenas.




## 3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

* Cobertura educativa se interpretará como el nivel de matrícula efectiva en población de edad escolar obligatoria (5 a 16 años), similar al enfoque anterior pero agregado a nivel de departamento, no municipio.

* Utilizaremos nuevamente la variable TASA_MATRICULACIÓN_5_16.

* Calcularemos el promedio histórico de esta tasa por departamento para establecer un ranking nacional.

In [44]:
query3 = '''
SELECT 
    g.departamento,
    ROUND(AVG(f.TASA_MATRICULACIÓN_5_16), 2) AS cobertura_promedio
FROM Fact_Matriculas f
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
GROUP BY g.departamento
HAVING COUNT(*) >= 5
ORDER BY cobertura_promedio DESC
LIMIT 10;
'''

df_cobertura = pd.read_sql_query(query3, conn)
df_cobertura


,departamento,cobertura_promedio
0,CESAR,96.93
1,SUCRE,95.53
2,MAGDALENA,94.66
3,QUINDÍO,93.88
4,META,91.26
5,ATLÁNTICO,89.46
6,CASANARE,89.25
7,CÓRDOBA,88.11
8,ANTIOQUIA,87.72
9,TOLIMA,87.65


In [52]:
import plotly.express as px
import pandas as pd

# Asegúrate que cobertura_promedio sea numérico
df_cobertura['cobertura_promedio'] = pd.to_numeric(df_cobertura['cobertura_promedio'], errors='coerce')

# Ordenar de mayor a menor
df_cobertura = df_cobertura.sort_values(by='cobertura_promedio', ascending=False)

# Crear gráfico interactivo tipo barras verticales
fig = px.bar(
    df_cobertura,
    x='departamento',
    y='cobertura_promedio',
    text='cobertura_promedio',
    color='cobertura_promedio',
    color_continuous_scale='Reds',
    labels={'departamento': 'Departamento', 'cobertura_promedio': 'Cobertura Promedio (%)'},
    title='🏫 Departamentos con mayor cobertura educativa (5 a 16 años)'
)

# Personalización estética
fig.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside',
    marker_line_color='darkred',
    marker_line_width=1
)

fig.update_layout(
    width=1100,
    height=600,
    xaxis_tickangle=45,
    xaxis_title='Departamento',
    yaxis_title='Cobertura promedio (%)',
    plot_bgcolor='white',
    font=dict(size=13),
    title_font_size=20,
    margin=dict(l=40, r=40, t=80, b=100),
    coloraxis_colorbar=dict(title='%')
)

fig.show()




Se tomó como base la variable **`TASA_MATRICULACIÓN_5_16`**, que expresa el porcentaje de niños, niñas y adolescentes matriculados en instituciones educativas, entre los 5 y 16 años de edad. Esta variable es clave porque representa el cumplimiento del derecho a la educación en su etapa obligatoria, y por lo tanto sirve como un indicador directo de cobertura educativa en cada departamento.

Se aplicó una consulta en lenguaje SQL que calcula el **promedio histórico de la tasa de matriculación por departamento**, agrupando los datos a lo largo del tiempo disponible en la base de datos. Para asegurar representatividad estadística, se filtraron únicamente aquellos departamentos con al menos cinco registros anuales.
El resultado se ordenó de mayor a menor para identificar los diez departamentos con mejores resultados.


* Los resultados reflejan que varios departamentos han alcanzado **niveles muy altos de cobertura**, con tasas promedio que superan el 95%. Esta cifra indica que en dichos territorios, **más del 95% de los menores entre 5 y 16 años están escolarizados**, lo que representa un éxito institucional en términos de acceso universal a la educación básica y media.

* Además, se identificaron patrones regionales donde los departamentos con mejor desempeño no siempre son los más grandes o más urbanos, lo cual demuestra que **la voluntad política, la inversión territorial y la eficiencia en la gestión educativa** pueden marcar la diferencia, más allá del tamaño o presupuesto del ente territorial.

* El hecho de que varios departamentos logren altos niveles de cobertura implica un sistema educativo sólido en cuanto a acceso. Sin embargo, esta cobertura no necesariamente garantiza calidad educativa, pero sí es una base necesaria para que los niños y adolescentes puedan desarrollar su trayectoria escolar con normalidad.

* Departamentos con cobertura inferior al 90% (aunque no aparecen en el ranking mostrado) podrían estar enfrentando desafíos como dificultades logísticas en zonas rurales, limitaciones presupuestales, o barreras culturales para el acceso a la educación.
